In [20]:
# 線形圧縮ベイズ
import numpy as np
import cv2
import sys
from tqdm import tqdm

In [21]:
Y = cv2.imread('./../img/onepiece01_luffy.png',0)
Y = cv2.resize(Y, (512, 512))
D = Y.shape[0]
N = Y.shape[1]

In [22]:
for M in [2, 4, 8, 16, 32, 64, 128]:
    # ハイパーパラメータ
    Sigma_omega = np.identity(M)*0.1
    Sigma_mu = np.identity(D)
    sigma_y = 31
    
    # 初期値
    m_mu = np.random.rand(D).reshape((D,1))
    m_omega = np.zeros((M,D))
    mu_x = np.random.rand(M,N)

    Sigma_mu_hat = np.random.rand(D,D)
    Sigma_oemga_hat = np.random.rand(M,M)
    Sigma_x_hat = np.random.rand(M,M)
    MAXITER = 20 #How many times will be repeted
    for i in tqdm(range(MAXITER)):

        #mu is reloated

        sum1 = np.zeros((D,1))
        for n in range(N):
            sum1 = sum1 + Y[:,n].reshape((D,1)) - np.dot(m_omega.T,mu_x[:,n].reshape((M,1)))

        m_mu = sigma_y**(-2) * np.dot(Sigma_mu_hat , sum1)
        Sigma_mu_hat = np.linalg.inv(N * sigma_y**(-2) * np.identity(D) + np.linalg.inv(Sigma_mu))

        #W is reloated

        for d in range(D):
            sum2 = np.zeros((M,1))
            sum3 = np.zeros((M,M))
            for n in range(N):

                sum2 = sum2 + (Y[d,n] - m_mu[d])*mu_x[:,n].reshape((M,1))
                sum3 = sum3 + np.dot(mu_x[:,n].reshape((M,1)),mu_x[:,n].reshape((1,M))) + Sigma_x_hat

            m_omega[:,d] = sigma_y**(-2) * np.dot(Sigma_oemga_hat , sum2).reshape((1,M))

        Sigma_oemga_hat = np.linalg.inv(sigma_y**(-2) * sum3 * np.linalg.inv(Sigma_omega))

        #X is reloated

        sum4 = np.zeros((M,M))
        for d in range(D):
            sum4 = sum4 + np.dot(m_omega[:,d].reshape((M,1)),m_omega[:,d].reshape((1,M))) + Sigma_oemga_hat
        Sigma_x_hat = np.linalg.inv(sigma_y**(-2)*sum4 + np.identity(M))

        for n in range(N):
            mu_x[:,n] = ( sigma_y**(-2)*Sigma_x_hat @ m_omega @ (Y[:,n].reshape((D,1)) - m_mu) ).reshape((1,M))

    reductioned_Y = m_omega.T @ mu_x + m_mu
    cv2.imwrite(f"./../img/M_{M}.jpg",reductioned_Y)


  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [04:48<1:31:23, 288.58s/it]


KeyboardInterrupt: 